<a href="https://colab.research.google.com/github/ChanapatC/Customer-Analytics/blob/main/Justdoit_review_opp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd


In [ ]:
!pip install pyLDAvis

In [ ]:
import gensim
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [ ]:
!pip install --upgrade pythainlp


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 17.6 MB/s eta 0:00:00


In [ ]:
import pythainlp

In [ ]:
df = pd.read_csv('JDI_review_oppenheimer_movie.csv')

In [ ]:
df.tail()

,review_id,comment
7,349,โนแลน คือคนที่ทำให้เชื่อทุกครั้ง ในสิ่งที่เล่า
8,907,ชอบเพราะแค่ดู เราก็แทบจะสัมผัสได้ทั้ง รูป รส ก...
9,899,นำเสรอได้น่าดูมากเลยคะ ดารานำรุ้ว่าเล่นดีตีบทแ...
10,474,Nolan หนังคุณภาพ ที่คนชอบดูหนังไม่ควรพลาด อีกเ...
11,616,คำพูดที่ว่าโลกยังไม่พร้อม ก็คงแบบเดียวกับ ai ล...


TOKENIZED

In [ ]:
stopwords = list(pythainlp.corpus.thai_stopwords())
removed_words = ['หนัง', ' ',]
screening_words = stopwords + removed_words

def tokenize_with_space (sentence):
  merged = ''
  words = pythainlp.word_tokenize(str(sentence), engine='newmm')
  for word in words:
    if word not in screening_words:
      merged = merged + ',' + word
  return merged [1:]

In [ ]:
df['review_tokenized'] = df['comment'].apply(lambda x: tokenize_with_space(x))

In [ ]:
df.tail()

,review_id,comment,review_tokenized
7,349,โนแลน คือคนที่ทำให้เชื่อทุกครั้ง ในสิ่งที่เล่า,"โน,แลน,คน,เล่า"
8,907,ชอบเพราะแค่ดู เราก็แทบจะสัมผัสได้ทั้ง รูป รส ก...,"ชอบ,ดู,แทบจะ,สัมผัส,รูป,รส,กลิ่น,เสียง,เหมือน,..."
9,899,นำเสรอได้น่าดูมากเลยคะ ดารานำรุ้ว่าเล่นดีตีบทแ...,"เส,รอ,น่าดู,ดารา,รุ้,เล่น,ดี,ตีบทแตก,เรื่อง,ดา..."
10,474,Nolan หนังคุณภาพ ที่คนชอบดูหนังไม่ควรพลาด อีกเ...,"Nolan,คุณภาพ,คน,ชอบ,ดู,พลาด,เรื่อง,นึง"
11,616,คำพูดที่ว่าโลกยังไม่พร้อม ก็คงแบบเดียวกับ ai ล...,"คำพูด,โลก,ai,มั้ง,สุดท้าย,กำเนิด,แปล,ให้ได้,ปล..."


**CREATE DICTIONARY**

In [ ]:
documents = df['review_tokenized'].to_list()
texts = [[text for text in doc.split(',')] for doc in documents]
dictionary = gensim.corpora.Dictionary(texts)

In [ ]:
print(dictionary.token2id.keys())

dict_keys(['ขาย', 'คับ', 'ชัดเจน', 'ตลาด', 'ตัวเอง', 'ทั่วโลก', 'ทำเงิน', 'ผู้กำกับ', 'ลายเซ็น', 'หา', 'แลน', 'โน', 'กำกับ', 'กี่', 'คน', 'ชื่อ', 'ดารา', 'น่าดู', 'ผู้ชม', 'ร่วมงาน', 'สนใจ', 'เก่ง', 'แทบ', 'การคัดเลือก', 'ดง', 'บท', 'ปฏิเสธ', 'ประทับใจ', 'ฝีมือ', 'พูดถึง', 'ฟี่', 'รับบท', 'เ', 'เมอร์', 'เรื่อง', 'เส', 'เหมาะกับ', 'ไม่ต้อง', 'cg', 'ความตั้งใจ', 'ความสมจริง', 'คุณภาพ', 'งาน', 'จิตวิญญาณ', 'ฉาก', 'ชอบ', 'ผลงาน', 'ภาพ', 'สัมผัส', 'เจ๋ง', 'ใส่', 'ให้ได้', 'Batnan', 'wow', 'การ์ตูน', 'ก้อ', 'ขนาด', 'คัว', 'ดู', 'ตอน', 'ถาม', 'ผม', 'มาจาก', 'ว้าว', 'สมจริง', 'สร้าง', 'หนังสือ', 'เบื่อ', 'เสมอ', 'แลน์', 'โจํกเกอร์', 'โดยเฉพาะ', 'blinders', 'peaky', 'การแสดง', 'คำตอบ', 'ชาย', 'ทึ่ง', 'อ.', 'แนะนำ', 'แน่นอน', 'ได้ดี', 'ไหม', '555', 'CGI', 'IMAX', 'ของจริง', 'ควรจะ', 'ตัอง', 'ต่างจังหวัด', 'นักแสดง', 'พ่อ', 'ระเบิด', 'อด', 'เบื้องหลัง', 'เสด็จ', 'โรง', 'เล่า', 'กลิ่น', 'คนดู', 'ดึง', 'ตรงนั้น', 'ทำ', 'พล็อต', 'ยืน', 'รส', 'รูป', 'สื่อสาร', 'หลุด', 'ออกมา', 'เกิดขึ้น', 'เสียง', 'เ

In [ ]:
gensim_corpus = [dictionary.doc2bow(text, allow_update=True) for text in texts]
word_frequencies = [[(dictionary[id], frequence) for id, frequence in couple] for couple in gensim_corpus]

**TOPIC MODELING**

In [ ]:
num_topics = 5
chunksize = 4000 # size of the doc looked at every pass
passes = 20 # number of passes through documents
iterations = 50
eval_every = 1 # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = dictionary[0] # This is only to "load" the dictionary.
id2word = dictionary.id2token

%time model = gensim.models.LdaModel(corpus=gensim_corpus, id2word=id2word, chunksize=chunksize, \
                                     alpha='auto', eta='auto',\
                                     iterations=iterations, num_topics=num_topics,\
                                     passes=passes, eval_every=eval_every)

CPU times: user 207 ms, sys: 1.04 ms, total: 208 ms
Wall time: 451 ms


In [ ]:
# Visualize the topic model
vis = pyLDAvis.gensim.prepare(model, gensim_corpus, dictionary, mds='mmds')
vis

/usr/local/lib/python3.10/dist-packages/sklearn/manifold/_mds.py:299: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.091622  0.111156       1        1  28.686944
2      0.014894 -0.159792       2        1  24.178578
3     -0.111812 -0.038007       3        1  21.845190
1     -0.070576  0.110443       4        1  18.443215
0      0.075872 -0.023800       5        1   6.846074, topic_info=        Term      Freq     Total Category  logprob  loglift
10       แลน  4.000000  4.000000  Default  30.0000  30.0000
14        คน  4.000000  4.000000  Default  29.0000  29.0000
45       ชอบ  3.000000  3.000000  Default  28.0000  28.0000
7   ผู้กำกับ  2.000000  2.000000  Default  27.0000  27.0000
30       ฟี่  1.000000  1.000000  Default  26.0000  26.0000
..       ...       ...       ...      ...      ...      ...
11        โน  0.065941  5.600321   Topic5  -5.3654  -1.7603
55       ก้อ  0.065941  1.230134   Topic5  -5.3654  -0.2446
69      แลน์  0.065940  1.230135   Topic5  -5.3654  -0.2446
64    สมจริง  0.065939  1.230136   Topic5  -5.3654  -0.2447
59       ตอน  0.065938  1.230137   Topic5  -5.3654  -0.2447

[227 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
83        3  0.852871      555
52        1  0.812921   Batnan
84        3  0.852871      CGI
85        3  0.852872     IMAX
130       1  0.812889    Nolan
...     ...       ...      ...
51        3  0.420143   ให้ได้
51        4  0.420143   ให้ได้
128       3  0.852872       ได
37        1  0.812889  ไม่ต้อง
129       3  0.852872    ไอรอน

[154 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 3, 4, 2, 1])

**PREDICT TOPICS**

In [ ]:
model.show_topic (1)

[('ชอบ', 0.047903854),
 ('ดู', 0.03293227),
 ('คนดู', 0.032931674),
 ('กำเนิด', 0.017964663),
 ('คำพูด', 0.017964663),
 ('ปล.', 0.017964661),
 ('ai', 0.017964661),
 ('โลก', 0.017964661),
 ('สิ่งใหม่', 0.017964661),
 ('แปล', 0.017964661)]

In [ ]:
df['topics'] = df['review_tokenized'].apply(lambda x: model.get_document_topics(dictionary.doc2bow(x.split(',')))[0][0])
df['score'] = df['review_tokenized'].apply(lambda x: model.get_document_topics(dictionary.doc2bow(x.split(',')))[0][1])


**GROUPING TOPIC AND SCORE**

In [ ]:
df.tail()

,review_id,comment,review_tokenized,topics,score
7,349,โนแลน คือคนที่ทำให้เชื่อทุกครั้ง ในสิ่งที่เล่า,"โน,แลน,คน,เล่า",2,0.960128
8,907,ชอบเพราะแค่ดู เราก็แทบจะสัมผัสได้ทั้ง รูป รส ก...,"ชอบ,ดู,แทบจะ,สัมผัส,รูป,รส,กลิ่น,เสียง,เหมือน,...",1,0.991306
9,899,นำเสรอได้น่าดูมากเลยคะ ดารานำรุ้ว่าเล่นดีตีบทแ...,"เส,รอ,น่าดู,ดารา,รุ้,เล่น,ดี,ตีบทแตก,เรื่อง,ดา...",3,0.992370
10,474,Nolan หนังคุณภาพ ที่คนชอบดูหนังไม่ควรพลาด อีกเ...,"Nolan,คุณภาพ,คน,ชอบ,ดู,พลาด,เรื่อง,นึง",4,0.979602
11,616,คำพูดที่ว่าโลกยังไม่พร้อม ก็คงแบบเดียวกับ ai ล...,"คำพูด,โลก,ai,มั้ง,สุดท้าย,กำเนิด,แปล,ให้ได้,ปล...",1,0.988092


In [ ]:
print (df)

    review_id                                            comment  \
0         545  ผู้กำกับแบบโนแลนหายากจริงๆนะคับ หนังของแกมีควา...   
1         811  เป็นผู้กำกับไม่กี่คน ที่ผู้ชมแทบไม่สนใจเลยว่าใ...   
2         879  สิ่งหนึ่งที่ประทับใจเรื่องนี้คือการคัดเลือกนัก...   
3         631  ชอบหนังโนแลนเพราะผู้กำกับใส่จิตวิญญาณและความตั...   
4         133  หนังโนแลน์ทำให้ผมว้าวได้เสมอ โดยเฉพาะBatnan โจ...   
5         793  ใครที่แบบคิดว่า เมอร์ฟี่จะแสดงได้ดีไหม peaky b...   
6         521  หนังของเสด็จพ่อโนแลนตัองดูในโรงให้ได้ครับ เบื้...   
7         349     โนแลน คือคนที่ทำให้เชื่อทุกครั้ง ในสิ่งที่เล่า   
8         907  ชอบเพราะแค่ดู เราก็แทบจะสัมผัสได้ทั้ง รูป รส ก...   
9         899  นำเสรอได้น่าดูมากเลยคะ ดารานำรุ้ว่าเล่นดีตีบทแ...   
10        474  Nolan หนังคุณภาพ ที่คนชอบดูหนังไม่ควรพลาด อีกเ...   
11        616  คำพูดที่ว่าโลกยังไม่พร้อม ก็คงแบบเดียวกับ ai ล...   

                                     review_tokenized  topics     score  
0   ผู้กำกับ,โน,แลน,หา,คับ,ตัวเอง,ลายเซ็น